In [1]:
siteURL = "https://www.theblueground.com/furnished-apartments-istanbul-turkey"
excelName = "py-bot-data.xlsx"

In [2]:
# Tarayıcıyı açar ve parametre olarak verilen URL yi ziyaret eder.

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import time

def getBrowser(url):
    options = Options()
    options.add_argument("--headless") 
    
    # ChromeDriver'ı otomatik olarak günceller.
    chromedriver_autoinstaller.install()  

    browser = webdriver.Chrome(options=options)
    browser.get(url)
    time.sleep(1)
    return browser

In [3]:
# Tarayıcıyı kapatır.

def closeBrowser(browser):
    browser.close()

In [4]:
from openpyxl import Workbook,load_workbook

# Seçilen excel hücresine veriyi yazar.

def writeExcel(data, row, column):
    wb = load_workbook(excelName)
    sheet = wb.active
    sheet.cell(row = row, column = column).value = data
    wb.save(excelName)

In [5]:
# BeautifulSoup kütüphanesini kullanarak sayfanın html içeriğini ayrıştırır.

from bs4 import BeautifulSoup

def getSoup(browser):
    soup = BeautifulSoup(browser.page_source, "html.parser")
    return soup

In [6]:
from selenium.webdriver.common.by import By

def sendReqBlueground(url):
   
    browser = getBrowser(url)
    
    pageSayac = 0
    # Sayfa sonundaki butona tıklayarak tek bir sayfa üzerinde sitedeki tüm evleri ekrana getirir.
    while(True):
        try:
            pageSayac = pageSayac + 1
            browser.execute_script("arguments[0].click();", browser.find_element(By.CLASS_NAME, "ui-button--large"))
            time.sleep(1)
            print(pageSayac)
        except:
            print("bitti")
            break
        
    listHomes = browser.find_elements(By.CLASS_NAME, 'property--TRY')
    print(len(listHomes))
    
    # Dataframe'mizin column adlarını yazdırır.
    writeExcel("ID", 1, 1)
    writeExcel("NAME", 1, 2)
    writeExcel("PRICE FOR MONTH", 1, 3)
    writeExcel("BEDROOM COUNT", 1, 4)
    writeExcel("BATHROOM COUNT", 1, 5)
    writeExcel("M2", 1, 6)
    writeExcel("FLOOR", 1, 7)
    writeExcel("GUESTS", 1, 8)
    writeExcel("CİTY", 1, 9)
    writeExcel("AREA", 1, 10)
    writeExcel("DISTRICT", 1, 11)
    writeExcel("LOCATION LAT", 1, 12)
    writeExcel("LOCATION LNG", 1, 13)
    writeExcel("EVENTS", 1, 14)
    writeExcel("SMOKE", 1, 15)
    writeExcel("PETS", 1, 16)
    

    # Tek tek tüm evlerin detay sayfalarına girip evlerle ilgili dataları çeker.
    sayac = 2
    for home in listHomes:
        try:
            browser.execute_script("arguments[0].click();", home)
            tabs = browser.window_handles
            browser.switch_to.window(tabs[(len(tabs) - 1)])
            time.sleep(1)
        except Exception as e:
            continue

        try:
            bluegroundGetDatasFromBuildingDetail(browser, sayac)
        except Exception as e:
            continue
        
        try:
            browser.close()
            browser.switch_to.window(tabs[0])
        except Exception as e:
            continue
        
        time.sleep(1)
        print(sayac - 1, ". ev alındı!")
        print("******************")
        sayac = sayac + 1
        
    
    closeBrowser(browser)
    print("---------")
    print("---------")



In [7]:
def bluegroundGetDatasFromBuildingDetail(buildingBrowser: webdriver.Chrome, row):
    
    soup = getSoup(buildingBrowser)
    
    buildingName = buildingBrowser.execute_script("return Blueground.pageData.property.name")
    buildingPriceForMonth = buildingBrowser.execute_script("return Blueground.pageData.property.lowestRent.amount")
    buildingBedroomCount = buildingBrowser.execute_script("return Blueground.pageData.property.bedrooms")
    buildingBathroomCount = buildingBrowser.execute_script("return Blueground.pageData.property.amenities.main[1]['value']")
    m2FloorGuestsDetails = soup.find_all("span", {"class": "property__amenities-list-item"})
    buildingM2 = m2FloorGuestsDetails[0].text
    buildingFloor = m2FloorGuestsDetails[1].text
    buildingGuests = m2FloorGuestsDetails[2].text
    buildingID = m2FloorGuestsDetails[4].text
    print(buildingID)
    buildingCity = buildingBrowser.execute_script("return Blueground.pageData.property.address.city")
    buildingArea = buildingBrowser.execute_script("return Blueground.pageData.property.address.area")
    buildingDistrict = buildingBrowser.execute_script("return Blueground.pageData.property.address.level2")
    buildingLat = buildingBrowser.execute_script("return Blueground.pageData.property.address.lat")
    buildingLng = buildingBrowser.execute_script("return Blueground.pageData.property.address.lng")
    buildingRules = buildingBrowser.execute_script("return Blueground.pageData.property.rules")
    buildingRuleEvents = "İzin Var" if buildingRules["eventsAllowed"] else "İzin Yok"
    buildingRuleSmoke = "İzin Var" if buildingRules["smokingAllowed"] else "İzin Yok"
    buildingRulePets = "İzin Var" if buildingRules["petsAllowed"] else "İzin Yok"

    writeExcel(buildingID, row, 1)
    writeExcel(buildingName, row, 2)
    writeExcel(buildingPriceForMonth, row, 3)
    writeExcel(buildingBedroomCount, row, 4)
    writeExcel(buildingBathroomCount, row, 5)
    writeExcel(buildingM2, row, 6)
    writeExcel(buildingFloor, row, 7)
    writeExcel(buildingGuests, row, 8)
    writeExcel(buildingCity, row, 9)
    writeExcel(buildingArea, row, 10)
    writeExcel(buildingDistrict, row, 11)
    writeExcel(buildingLat, row, 12)
    writeExcel(buildingLng, row, 13)
    writeExcel(buildingRuleEvents, row, 14)
    writeExcel(buildingRuleSmoke, row, 15)
    writeExcel(buildingRulePets, row, 16)

    time.sleep(2)



In [8]:
sendReqBlueground(siteURL)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
bitti
875
ID: 958
1 . ev alındı!
******************
ID: 910
2 . ev alındı!
******************
ID: 660
3 . ev alındı!
******************
ID: 1346
4 . ev alındı!
******************
ID: 1118
5 . ev alındı!
******************
ID: 856
6 . ev alındı!
******************
ID: 914
7 . ev alındı!
******************
ID: 1015
8 . ev alındı!
******************
ID: 993
9 . ev alındı!
******************
ID: 1352
10 . ev alındı!
******************
ID: 308
11 . ev alındı!
******************
ID: 1337
12 . ev alındı!
******************
ID: 1336
13 . ev alındı!
******************
ID: 995
14 . ev alındı!
******************
ID: 1027
15 . ev alındı!
******************
ID: 1315
16 . ev alındı!
******************
ID: 1150
17 . ev alındı!
******************
ID: 1243
18 . ev alındı!
******************
ID: 401
19 . ev alındı!
********

ID: 946
184 . ev alındı!
******************
ID: 932
185 . ev alındı!
******************
ID: 1007
186 . ev alındı!
******************
ID: 26
187 . ev alındı!
******************
ID: 961
188 . ev alındı!
******************
ID: 571
189 . ev alındı!
******************
ID: 818
190 . ev alındı!
******************
ID: 8
191 . ev alındı!
******************
ID: 9
192 . ev alındı!
******************
ID: 1126
193 . ev alındı!
******************
ID: 832
194 . ev alındı!
******************
ID: 130
195 . ev alındı!
******************
ID: 4
196 . ev alındı!
******************
ID: 1181
197 . ev alındı!
******************
ID: 178
198 . ev alındı!
******************
ID: 556
199 . ev alındı!
******************
ID: 1251
200 . ev alındı!
******************
ID: 1033
201 . ev alındı!
******************
ID: 1043
202 . ev alındı!
******************
ID: 919
203 . ev alındı!
******************
ID: 859
204 . ev alındı!
******************
ID: 1172
205 . ev alındı!
******************
ID: 337
206 . ev alındı!
*******

ID: 1141
369 . ev alındı!
******************
ID: 668
370 . ev alındı!
******************
ID: 808
371 . ev alındı!
******************
ID: 690
372 . ev alındı!
******************
ID: 1019
373 . ev alındı!
******************
ID: 766
374 . ev alındı!
******************
ID: 478
375 . ev alındı!
******************
ID: 1105
376 . ev alındı!
******************
ID: 953
377 . ev alındı!
******************
ID: 831
378 . ev alındı!
******************
ID: 1216
379 . ev alındı!
******************
ID: 1173
380 . ev alındı!
******************
ID: 373
381 . ev alındı!
******************
ID: 1189
382 . ev alındı!
******************
ID: 1201
383 . ev alındı!
******************
ID: 764
384 . ev alındı!
******************
ID: 1122
385 . ev alındı!
******************
ID: 980
386 . ev alındı!
******************
ID: 402
387 . ev alındı!
******************
ID: 952
388 . ev alındı!
******************
ID: 129
389 . ev alındı!
******************
ID: 1291
390 . ev alındı!
******************
ID: 1233
391 . ev alınd